<p>
<img src="./imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **Algoritmo de Viola-Jones**

https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf

El **algoritmo de Viola–Jones** (2001–2004) es un método clásico de **detección rápida de objetos** —famosamente, **caras frontales**— en imágenes. Su éxito se basa en combinar tres ideas:

1. **Características tipo Haar (Haar-like features)**
   Rectángulos blancos/negros que miden diferencias de intensidad (bordes, líneas, centros más claros/osc.) dentro de una ventana. Cada característica es:

   $$
   f(x)=\sum_{\text{blanco}} I - \sum_{\text{negro}} I
   $$

   donde $I$ es la imagen.

2. **Imagen integral (integral image / summed-area table)**
   Estructura que permite calcular la suma de intensidades de cualquier rectángulo en O(1), haciendo viable evaluar miles de características por ventana muy rápido.

3. **AdaBoost + cascada de clasificadores**

   * **AdaBoost** selecciona, de entre decenas de miles de características, unas pocas muy discriminativas y construye un **clasificador fuerte** como combinación ponderada de **clasificadores débiles** (umbrales sobre una sola característica).
   * **Cascada**: organiza varios clasificadores de menor a mayor complejidad. La mayoría de ventanas se descartan en etapas tempranas con coste mínimo; sólo las candidatas pasan a etapas más estrictas. Esto permite tiempo real en CPUs modestas.

## **¿Cómo detecta?**

* Se recorre la imagen con una **ventana deslizante** a **múltiples escalas**.
* Para cada ventana:

  1. Se calculan rápidamente las características con la imagen integral.
  2. Se evalúa etapa a etapa la **cascada**; si la ventana falla en alguna, se descarta.
  3. Las ventanas que superan todas las etapas son detecciones; luego se aplica **Non-Maximum Suppression (NMS)** para unificar solapes.

## **Entrenamiento (resumen)**

* Conjuntos de **positivos** (caras) y **negativos** (no-cara).
* AdaBoost itera: elige la mejor característica+umbral (débil) que minimiza el error ponderado; actualiza pesos; combina débiles → **fuerte**.
* Se apila una serie de clasificadores fuertes en **cascada**, ajustando cada etapa para alcanzar alta detección con bajos falsos positivos acumulados.

## **Ventajas**

* **Rápido** y apto para **tiempo real** sin GPU.
* Implementaciones maduras (p. ej., `cv2.CascadeClassifier`).

## **Limitaciones**

* Funciona mejor para caras frontales con buena iluminación.
* Sensible a variaciones de pose** (perfil, inclinación), oclusiones y cambios fuertes de iluminación.
* Hoy está superado por CNNs (HOG+SVM, DPM y, sobre todo, detectores modernos tipo Faster/Mask R-CNN, SSD, YOLO), que son más robustos.

In [ ]:
import cv2

# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# face_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
# face_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml')


# cap = cv2.VideoCapture("people_walking2.mp4")
cap = cv2.VideoCapture(1)

while(1):
 
    # Take each frame
    _, frame = cap.read()
    frame = cv2.resize(frame, (400, 300))
     
    # Convert to HSV for simpler calculations
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(frame)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    cv2.imshow('frame',frame)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()


Es posible descargar más detectores en:

https://github.com/opencv/opencv/tree/master/data/haarcascades

https://towardsdatascience.com/face-detection-with-haar-cascade-727f68dafd08

https://www.youtube.com/watch?v=LsK-xG1cLYA

Se escogen diferentes patrones que se intentarán buscar en la imagen en diferentes posiciones y escalas. Cada patrón se considera como un clasificador débil. La unión de muchos de estos clasificadores mediante boosting convierte a este método en un clasificador robusto.

<img src="haar8.png" width="30%">

<img src="haar9.jpeg" width="30%">

Una vez aplicado el patrón en un lugar concreto de la imagen, se lleva a cabo el cálculo de la comparación.

<img src="haar2.webp" width="100%">

Cada patrón se buscar a lo largo y ancho de la imagen.

<img src="haar3.gif" width="40%">

Este proceso es muy costoso computacionalmente así que se crea una iamgen integral que reducirá muchísimo los cálculos.

<img src="haar4.gif" width="100%">

<img src="haar5.webp" width="100%">

Otro truco para mejorar la velocidad consiste en pasar los patrones en diferentes estapas, el algoritmo original usa 38 etapas. Cuando una no responde positivamente la subventana se descarta y no se considera que contenga una cara.

<img src="haar6.gif" width="70%">

Cuando una subventana pasa todos los patrones se considera que ha detectado una cara.

<img src="haar7.gif" width="40%">



## **Ejercicios**

- Adapta el algoritmo de detección de caras a detección de personas (cuerpo entero). Usa el vídeo <code>people_walking.mp4</code> para probarlo.

- Desarrolla un sistema de detección de caras robusto que mantenga la atención sobre una cara sin perder su posición. Para ello, combina la función de detección de caras con la función de tracking de manera que, cuando la detección de caras falle, el tracking pueda seguir encuadrando la cara.